In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import ensemble
import pickle

In [3]:
df = pd.read_csv("address_with_coords.csv").dropna()
df["lon"] = df.point.apply(lambda x: float(x.split()[0]))
df["lat"] = df.point.apply(lambda x: float(x.split()[1]))
df = df[(df["lat"]>55.582238) & (df["lat"]<55.905471) & (df["lon"]>37.372692) & (df["lon"]<37.851569)]
df = df[["timestamp", "lon", "lat"]]

df["timestamp"] = df.timestamp.apply(lambda x: pd.to_datetime(x, format="%Y-%m-%dT%H:%M:%S.%f"))
df["minute"] = df.timestamp.apply(lambda x: x.minute)
df["second"] = df.timestamp.apply(lambda x: x.second)
df["hour"] = df.timestamp.apply(lambda x: x.hour)
df["month"] = df.timestamp.apply(lambda x: x.month)
df["day"] = df.timestamp.apply(lambda x: x.day)
df["year"] = df.timestamp.apply(lambda x: x.year)
df["microsecond"] = df.timestamp.apply(lambda x: x.microsecond)
df = df.drop("timestamp", axis=1)
df.to_csv("mini_address_with_coords.csv", index=False)

In [43]:
df_train = pd.read_csv("mini_address_with_coords.csv")
df_train = df_train.head(10000)
df_train

,lon,lat,minute,second,hour,month,day,year,microsecond
0,37.619405,55.869647,38,1,12,5,10,2020,157000
1,37.664734,55.600359,28,14,13,5,22,2020,875000
2,37.664734,55.600359,6,11,16,5,14,2020,6000
3,37.524848,55.681757,27,42,17,5,14,2020,238000
4,37.603361,55.867556,31,57,12,5,16,2020,140000
...,...,...,...,...,...,...,...,...,...
9995,37.666019,55.893485,48,45,9,6,4,2020,224000
9996,37.665497,55.892668,50,31,9,6,23,2020,166000
9997,37.399327,55.710053,8,56,12,6,8,2020,973000
9998,37.449534,55.730015,36,29,11,6,11,2020,391000


In [44]:
#lon model

X = df_train.drop("lon", axis=1)
y = df_train[["lon"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.5, random_state=1)

# Set the parameters by cross-validation
tuned_parameters = {'n_estimators': [10], 'max_depth': [None, 2], 'min_samples_split': [2]}

# clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1, verbose=1)
clf = GridSearchCV(ensemble.RandomForestRegressor(), tuned_parameters, cv=2, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=2, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [45]:
lon_model = clf.best_estimator_

In [46]:
#lat model

X = df_train.drop("lat", axis=1)
y = df_train[["lat"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.5, random_state=1)

# Set the parameters by cross-validation
tuned_parameters = {'n_estimators': [10], 'max_depth': [None, 2], 'min_samples_split': [2]}

# clf = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1, verbose=1)
clf = GridSearchCV(ensemble.RandomForestRegressor(), tuned_parameters, cv=2, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=2, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [47]:
lat_model = clf.best_estimator_

In [48]:
with open('lon_model', 'wb') as f:
  pickle.dump(lon_model, open("lon_model", 'wb'))

with open('lat_model', 'wb') as f:
  pickle.dump(lat_model, open("lat_model", 'wb'))